# import libs

In [80]:
import ast
import pandas as pd
import numpy as np
from sklearn import preprocessing
import datetime

from pandas_gbq import read_gbq , to_gbq
import time
import hashlib

import json

from tqdm import tqdm

In [2]:
import plotly.express as px


# load data

In [344]:
df_eventlog = pd.read_csv('./data/real_gdp_to_deltadna_v2.csv')

df_eventlog.head(2)

,packages,eventName,userID,sessionID,eventUUID,eventTimestamp,eventParams,ingestTimestamp,clientIP,packageName
0,NaN,sensor,552c3978f3cecc46,5bd3bfd7-c36c-4b55-b773-4004c31b3e1c,074a2545-d6a7-4a75-876d-8945ce279645,2020-11-19 13:24:36.332000,"{""sensor-name"": ""sar_detector Wakeup"", ""senso...",2020-11-19 13:32:37.502000,78.56.89.146,ai.gosu.dataplatform.nodulus
1,NaN,sensor,552c3978f3cecc46,5bd3bfd7-c36c-4b55-b773-4004c31b3e1c,4c7f1a2e-5ac1-4fb2-b9da-7f3f71a9544c,2020-11-19 13:24:36.331000,"{""sensor-name"": ""NonUi Wakeup"", ""sensor-type""...",2020-11-19 13:32:37.502000,78.56.89.146,ai.gosu.dataplatform.nodulus


# quick analysis

In [345]:
df_eventlog.groupby(['eventName']).count().sort_values(by = ['userID'] , ascending=False).head(15)

,packages,userID,sessionID,eventUUID,eventTimestamp,eventParams,ingestTimestamp,clientIP,packageName
eventName,,,,,,,,,
sensor,0,23817,23817,23817,23817,23817,13752,13491,11256
broadcast,0,4265,4265,4265,4265,4265,2568,2348,1556
notifications,0,981,981,981,981,893,981,981,981
notification-channels,0,843,843,843,843,841,843,843,843
lifecycle-start,0,575,575,575,575,0,293,271,215
lifecycle-resume,0,574,574,574,574,0,293,271,215
info,0,502,502,502,502,502,238,221,155
lifecycle-create,0,478,478,478,478,0,217,200,153
Application.onCreate,0,368,368,368,368,0,109,92,57


In [355]:
user = '552c3978f3cecc46'
event = 'sensor'

df_test = df_eventlog[(df_eventlog['userID'] == user)
           &(df_eventlog['eventName'] == event)][:]


df_test['is_event'] = 1
df_test.head(3)


,packages,eventName,userID,sessionID,eventUUID,eventTimestamp,eventParams,ingestTimestamp,clientIP,packageName,is_event
0,NaN,sensor,552c3978f3cecc46,5bd3bfd7-c36c-4b55-b773-4004c31b3e1c,074a2545-d6a7-4a75-876d-8945ce279645,2020-11-19 13:24:36,"{""sensor-name"": ""sar_detector Wakeup"", ""senso...",2020-11-19 13:32:37.502000,78.56.89.146,ai.gosu.dataplatform.nodulus,1
1,NaN,sensor,552c3978f3cecc46,5bd3bfd7-c36c-4b55-b773-4004c31b3e1c,4c7f1a2e-5ac1-4fb2-b9da-7f3f71a9544c,2020-11-19 13:24:36,"{""sensor-name"": ""NonUi Wakeup"", ""sensor-type""...",2020-11-19 13:32:37.502000,78.56.89.146,ai.gosu.dataplatform.nodulus,1
2,NaN,sensor,552c3978f3cecc46,5bd3bfd7-c36c-4b55-b773-4004c31b3e1c,c0039e14-b85c-404a-8557-254377bb3f04,2020-11-19 13:24:36,"{""sensor-name"": ""tcs3701 Non-wakeup"", ""sensor...",2020-11-19 13:32:37.502000,78.56.89.146,ai.gosu.dataplatform.nodulus,1


In [356]:

fig = px.scatter(df_test, x="eventTimestamp", y="is_event", color="is_event",

                )

fig.show()

In [357]:
#check event dict params
print(df_test.index[:10])

#df = df_test['eventParams'].loc[500:563].apply(lambda x: ast.literal_eval(x))
df = df_test['eventParams'].loc[400:2000].apply(lambda x: json.loads(x.replace("'", "\"")))

df = pd.DataFrame(list(df))

goodevents = [4 , 3]

df



Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')


,sensor-name,sensor-type,sensor-string-type,events
0,Game Rotation Vector Non-wakeup,15,android.sensor.game_rotation_vector,"[{'timestamp': 1605791428840, 'values': [0, 0,..."
1,Aod Wakeup,33171029,xiaomi.sensor.aod,"[{'timestamp': 1605791428651, 'values': [4, 0,..."
2,tcs3701 Ambient Light Sensor Wakeup,5,android.sensor.light,"[{'timestamp': 1605791428644, 'values': [67.56..."
3,NonUi Non-wakeup,33171027,xiaomi.sensor.nonui,"[{'timestamp': 1605791428572, 'values': [0, 0,..."
4,har_wakeup,33171070,xiaomi.sensor.har,"[{'timestamp': 1605791429317, 'values': [0, 0...."
...,...,...,...,...
1424,tcs3701 Non-wakeup,33171007,xiaomi.sensor.ambientlight.factory,"[{'timestamp': 1605787960832, 'values': [150.5..."
1425,orientation Non-wakeup,3,android.sensor.orientation,"[{'timestamp': 1605787960606, 'values': [288.7..."
1426,NonUi Non-wakeup,33171027,xiaomi.sensor.nonui,"[{'timestamp': 1605787960500, 'values': [0, 0,..."
1427,NonUi Wakeup,33171027,xiaomi.sensor.nonui,"[{'timestamp': 1605787960500, 'values': [0, 0,..."


# preprocessing data

In [358]:
active_users = ['c525bf952aaec6fb','552c3978f3cecc46','34ef6b4d1e81efc6']
df_eventlog = df_eventlog[(df_eventlog['userID'].isin(active_users))]

df_eventlog['eventTimestamp'] = df_eventlog['eventTimestamp'].astype(str).str[:19]
df_eventlog['eventTimestamp'] = pd.to_datetime(df_eventlog['eventTimestamp'])

# rule based model

In [359]:
user = '552c3978f3cecc46'
df_user = df_eventlog[(df_eventlog['userID'] == user)][:]


df_user['is_notify'] = 0


df_user.sort_values(by = 'eventTimestamp' , inplace = True)

df_user.set_index('eventTimestamp' , inplace = True)

df_user = df_user[['eventName'  , 'eventParams' , 'is_notify']][:]

#test period
df_user = df_user.loc[:'2020-11-04'][:]

display(df_user.head(2))

display(df_user.tail(2))

print(df_user.shape)



,eventName,eventParams,is_notify
eventTimestamp,,,
2020-10-29 11:56:17,info,"{""name"": ""MI CC9 Pro"", ""model"": ""MI CC9 Pro"", ...",0
2020-10-29 11:56:22,sensors,"{""NonUi Non-wakeup"": [0, 0, 0, 0, 0, 0, 0, 0,...",0


,eventName,eventParams,is_notify
eventTimestamp,,,
2020-11-04 22:55:52,sensor,"{""sensor-name"": ""fod Wakeup"", ""sensor-type"": ...",0
2020-11-04 22:55:52,sensor,"{""sensor-name"": ""tcs3701 Non-wakeup"", ""sensor...",0


(2134, 3)


In [360]:
print(df_user[(df_user['eventName'] == 'broadcast')].head(2))

                     eventName  \
eventTimestamp                   
2020-10-29 19:17:09  broadcast   
2020-10-29 19:22:52  broadcast   

                                                           eventParams  \
eventTimestamp                                                           
2020-10-29 19:17:09        {"action": "android.net.wifi.STATE_CHANGE"}   
2020-10-29 19:22:52  {"action": "android.net.wifi.STATE_CHANGE", "n...   

                     is_notify  
eventTimestamp                  
2020-10-29 19:17:09          0  
2020-10-29 19:22:52          0  


# important factors

In [361]:



'''
summary: notify decision for check window
input: dict_notify = {'broadcast' : 0,
                   'notification-channels' : 0,
                   'lifecycle-start' : 0,
                   'lifecycle-resume' : 0,
                   'lifecycle-create' : 0
                  }
output: 0
'''
def check_notify(dict_notify):
    is_notify = 0
    
    is_notify += dict_notify['broadcast']
    is_notify += dict_notify['notification-channels']
    is_notify += dict_notify['lifecycle-start']
    is_notify += dict_notify['lifecycle-resume']
    is_notify += dict_notify['lifecycle-create']
    
    return is_notify


'''
summary: check goodevents at window
input: eventName = broadcast
        df =                     %eventName  \
            eventTimestamp                                                           
            2020-10-29 19:17:09  broadcast   
            2020-10-29 19:22:52  broadcast   

                               eventParams  \
            eventTimestamp                                                           
            2020-10-29 19:17:09        {"action": "android.net.wifi.STATE_CHANGE"}   
            2020-10-29 19:22:52  {"action": "android.net.wifi.STATE_CHANGE", "n...   

                                 is_notify  
            eventTimestamp                  
            2020-10-29 19:17:09          0  
            2020-10-29 19:22:52          0  

output: 0
'''
def goodevents(df , eventName):
    is_goodevent = 0
    
    dict_all_goodevents = {'broadcast' : {'action':['android.intent.action.DREAMING_STOPPED' 
                                                    , 'android.intent.action.HEADSET_PLUG' , 'android.intent.action.SCREEN_ON'
                                                   ,'android.net.conn.CONNECTIVITY_CHANGE','android.net.wifi.STATE_CHANGE'
                                                   ]
                                         },
                           'notification-channels' : {'importance': [4 , 3]
                                                     },
                           'lifecycle-start' : {'any':['any']
                                               },
                           'lifecycle-resume' : {'any':['any']
                                               },
                           'lifecycle-create' : {'any':['any']
                                               }
                      
                      }
    check_goodevents = dict_all_goodevents[eventName]
    
    try:
        df = df['eventParams'].apply(lambda x: json.loads(x.replace("'", "\"")))
        df = pd.DataFrame(list(df))
        
        for param in check_goodevents.keys():
        
            if len(np.intersect1d(df[param].values , check_goodevents[param])) > 0:
                is_goodevent = 1
        
    except:
        is_goodevent = 1
    

    return is_goodevent

'''
summary: window rule based stat model for last N second to make decision - >0 notify , 0 not notify
input:  df =                     %eventName  \
            eventTimestamp                                                           
            2020-10-29 19:17:09  broadcast   
            2020-10-29 19:22:52  broadcast   

                               eventParams  \
            eventTimestamp                                                           
            2020-10-29 19:17:09        {"action": "android.net.wifi.STATE_CHANGE"}   
            2020-10-29 19:22:52  {"action": "android.net.wifi.STATE_CHANGE", "n...   

                                 is_notify  
            eventTimestamp                  
            2020-10-29 19:17:09          0  
            2020-10-29 19:22:52          0  
output: 0
'''
def window_stats(df):
    
    dict_notify = {'broadcast' : 0,
                   'notification-channels' : 0,
                   'lifecycle-start' : 0,
                   'lifecycle-resume' : 0,
                   'lifecycle-create' : 0
                  }
    
    for f in dict_notify.keys():
        is_check = True if df[(df['eventName'] == f)].shape[0] > 0 else False
        if is_check:
            df_goodevents = df[(df['eventName'] == f)][:]
            is_f_notify = goodevents(df_goodevents , f)   
            dict_notify[f] = is_f_notify
        
    
    
    is_notify = check_notify(dict_notify)
    return is_notify

if __name__ == '__main__':
    #define seconds step for window check
    N_sec = 5

    dt_start = df_user.index.min()
    dt_end = df_user.index.max()

    dt_cur = dt_start



    while dt_cur <= dt_end:

        if df_user.loc[dt_cur:dt_next].shape[0] == 0:
            dt_cur = df_user[(df_user.index > dt_cur)].index.min()

        dt_next = dt_cur + datetime.timedelta(0,N_sec)


        print(f"{dt_cur} {  round(df_user.loc[dt_start:dt_cur].shape[0] / df_user.shape[0] , 4) }", end = '\r')

        df_user_part = df_user.loc[dt_cur:dt_next][:]

        is_notify = window_stats(df_user_part)
        df_user.loc[dt_cur:dt_next , 'is_notify'] = is_notify


        dt_cur = dt_next
    
    


In [362]:
#plot 

df = df_user.reset_index()

fig = px.scatter(df, x="eventTimestamp", y="is_notify", color="is_notify",

                )

fig.show()